In [14]:
import sys,operator
from math import log
from math import sqrt
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize

V=5001
U=100001

In [15]:
def findSim(matrix, word):
    global reverseVocab
    idd=reverseVocab[word]
    vals={}
    compared=sqrt((np.inner(matrix[idd], matrix[idd])))
    for i in range(1,V):
        sim=np.inner(matrix[idd], matrix[i])/(compared * sqrt(np.inner(matrix[i], matrix[i])))
        vals[i]=sim

    sorted_x = sorted(vals.items(), key=operator.itemgetter(1), reverse=True)
    for (k,v) in sorted_x[:10]:
        print "%.3f\t%s" % (v, vocab[k])


In [16]:
def readWords(filename):
    global reverseVocab, vocba
    file=open(filename)
    for line in file:
        cols=line.rstrip().split("\t")
        wordId=int(cols[0])
        word=cols[1]
        reverseVocab[word]=wordId
        vocab[wordId]=word
    file.close()

In [17]:
idfs=[0]*V
def readIdfs(filename):
    file=open(filename)
    for line in file:
        cols=line.rstrip().split("\t")
        wordId=int(cols[0])
        idf=float(cols[1])
        idfs[wordId]=idf
    file.close()

In [18]:
def readData(filename):
    file=open(filename)
    i=0
    for line in file:
        if (i % 100000) == 0:
            print i
        i+=1
        cols=line.rstrip().split(",")
        userId=int(cols[0])
        wordId=int(cols[1])
        count=int(cols[2])
        counts[wordId,userId]=count*log(float(U)/(1+idfs[wordId]))
    file.close()


In [19]:
def proc(k):	
    (u, s, v) = svds(counts, k=k)
    return (u, s, v)

In [20]:
vocab=[None]*V
reverseVocab={}
counts=sparse.dok_matrix((V,U))


In [21]:
readWords("unigrams.txt")

In [22]:
readIdfs("idf.txt")

In [23]:
print "reading data"
readData("data.txt")

reading data
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


In [24]:
counts=counts.tocsr()

In [13]:
print "calculating SVD"
(u,s,v)=proc(100)
print u.shape, s.shape, v.shape

calculating SVD
(5001, 100) (100,) (100, 100001)


In [14]:
def query(matrix1, q):
    print "most related terms (SVD)"
    findSim(matrix1,q)   

In [1]:
q="runner"
query(np.mat(u)*np.diag(s), q)

NameError: name 'query' is not defined